In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from tensorflow.contrib.layers import fully_connected, batch_norm, dropout
from tensorflow.contrib.framework import arg_scope
import tensorflow as tf
import random

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
tf.set_random_seed(777) # reproducibility

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [6]:
# parameters
learning_rate = 0.01 # we can use large learning rate using Batch Normalization
training_epochs = 15
batch_size = 100
keep_prob = 0.7

In [7]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
train_mode = tf.placeholder(tf.bool, name = 'train_mode')

In [8]:
# layer output size
hidden_output_size = 512
final_output_size = 10

In [9]:
xavier_init = tf.contrib.layers.xavier_initializer()
bn_params = {
    'is_training': train_mode,
    'decay': 0.9,
    'updates_collections': None
}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [10]:
# We can build short code using 'arg_scope' to avoid duplicate code
# same function with different arguments
with arg_scope([fully_connected],
               activation_fn = tf.nn.relu,
               weights_initializer = xavier_init,
               biases_initializer = None,
               normalizer_fn = batch_norm,
               normalizer_params = bn_params):
  hidden_layer1 = fully_connected(X, hidden_output_size, scope = "h1")
  h1_drop = dropout(hidden_layer1, keep_prob, is_training = train_mode)
  hidden_layer2 = fully_connected(h1_drop, hidden_output_size, scope = "h2")
  h2_drop = dropout(hidden_layer2, keep_prob, is_training = train_mode)
  hidden_layer3 = fully_connected(h2_drop, hidden_output_size, scope = "h3")
  h3_drop = dropout(hidden_layer3, keep_prob, is_training = train_mode)
  hidden_layer4 = fully_connected(h3_drop, hidden_output_size, scope = "h4")
  h4_drop = dropout(hidden_layer4, keep_prob, is_training = train_mode)
  hypothesis = fully_connected(h4_drop, final_output_size, activation_fn = None, scope = "hypothesis")

Instructions for updating:
Please use `layer.__call__` method instead.


In [11]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = hypothesis, labels = Y
))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
# train my model
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples / batch_size)

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    feed_dict_train = {X: batch_xs, Y: batch_ys, train_mode: True}
    feed_dict_cost = {X: batch_xs, Y: batch_ys, train_mode: False}
    opt = sess.run(optimizer, feed_dict = feed_dict_train)
    c = sess.run(cost,feed_dict = feed_dict_cost)
    avg_cost += c / total_batch
  
  print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 0.385370338
[Epoch:    2] cost = 0.331304152
[Epoch:    3] cost = 0.322650413
[Epoch:    4] cost = 0.315587626
[Epoch:    5] cost = 0.311579854
[Epoch:    6] cost = 0.310106164
[Epoch:    7] cost = 0.307810989
[Epoch:    8] cost = 0.305172341
[Epoch:    9] cost = 0.305305323
[Epoch:   10] cost = 0.303928513
[Epoch:   11] cost = 0.302796236
[Epoch:   12] cost = 0.301477652
[Epoch:   13] cost = 0.303861873
[Epoch:   14] cost = 0.302222298
[Epoch:   15] cost = 0.301056695
Learning Finished!


In [14]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy,
                            feed_dict = {X: mnist.test.images, Y: mnist.test.labels, train_mode: False}))

Accuracy: 0.9848


In [15]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(hypothesis, 1),
                          feed_dict = {X: mnist.test.images[r:r + 1], train_mode: False}))

Label:  [0]
Prediction:  [0]
